<a href="https://colab.research.google.com/github/heitorabqg/datascientist/blob/master/PSI5892_FashionMNIST_1128.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Para abrir o notebook no Google Colab, altere o domínio `github.com` para `githubtocolab.com`

<div class="alert alert-block alert-danger">
Para praticar programação, é importante que você erre, leia as mensagens de erro e tente corrigí-los.
    
Dessa forma, no Google Colab, é importante que você DESATIVE OS RECURSOS DE AUTOCOMPLETAR:

- Menu Ferramentas -> Configurações
- Na janela que é aberta:
  - Seção Editor -> Desativar "Mostrar sugestões de preenchimento de código com base no contexto"
  - Seção Assistência de IA -> Desabilitar itens

Na versão em inglês:

- Menu Tools -> Settings
- Na janela que é aberta:
  - Seção Editor -> Desativar "Show context-powered code completions"
  - Seção AI Assistance -> Desabilitar itens
</div>

# PSI5892 - Aula de Exercícios

# MLP com PyTorch

Neste exercício vamos treinar uma rede MLP usando o *framework* PyTorch, para a solução de um problema de classificação usando o banco de dados [Fashion MNIST](https://arxiv.org/abs/1708.07747), que contém 70000 imagens 28x28 de peças de vestuário distribuídas em 10 classes, divididas em um conjunto de treinamento com 60000 imagens e um de teste com 10000.

Os dados estão disponíveis na biblioteca `torchvision` e os objetos `DataLoader` podem ser criados com:

``` python
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

from torchvision import datasets, transforms

dir_data = "~/temp"

train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST(
        dir_data,
        train=True,
        download=True,
        transform=transforms.Compose(            
            [transforms.ToTensor()]
        ),
    ),
    batch_size=Nb,
    shuffle=True,
)

test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST(
        dir_data,
        train=False,
        transform=transforms.Compose(            
            [transforms.ToTensor()]
        ),
    ),
    batch_size=Nb_test,
    shuffle=True,
)
```

Vale notar alguns detalhes sobre o código anterior:

 - o `DataLoader` de treinamento é criado com `train=True` e o de teste, com `train=False`, o que garante que não haja dados em comum entre os dois conjuntos;
 - `Nb` e `Nb_test` representam os tamanhos dos mini *batches* de treino e teste;
 - É feita a configuração de uma transformação de dados ao carregá-los. Para isso, é criado um objeto do tipo `transforms.Compose`, que permite encadear uma série de transformações a serem aplicadas às imagens, durante o carregamento. Nesse caso, a transformação tem uma única etapa que consistem em converter os valores obtidos para um tensor do PyTorch.

Para ver algumas imagens do banco de dados usando o DataLoader criado, pode ser utilizado o seguinte código:

``` python
plt.figure(figsize=(16, 6))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    image, _ = train_loader.dataset.__getitem__(i)
    plt.imshow(image.squeeze().numpy())
    plt.axis('off');
```


# Exercício 1

Implemente uma rede MLP para classificação de imagens do conjunto Fashion MNIST usando o PyTorch. Lembre-se que trata-se de um problema de classificação multiclasse e utilize a arquitetura mais adequada.

No caso de usar a entropia cruzada, vale notar que a função custo `CrossEntropyLoss` espera comparar um vetor de $C$ posições com um número de $0$ a $C-1$, conforme descrito na [documentação](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html). Além disso, é esperado que os elementos do vetor representem a *evidência*, ou seja os valores chamados de *logits*, que não são normalizados e podem valer de $-\infty$ a $\infty$. Por isso, na saída da rede, não é usada a função *Softmax*.

Por fim, avalie o modelo treinado em termos de acurácia (número de acertos dividido pelo número de testes) e busque variar os hiperparâmetros da rede a fim de obter uma acurácia próxima a 85%.

## Resolução

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader



Nb = 100

Nb_test = 100
lr = 0.001
epochs = 20


dir_data = "./data"

transform = transforms.Compose([transforms.ToTensor()])

train_loader = DataLoader(
    datasets.FashionMNIST(
        dir_data,
        train=True,
        download=True,
        transform=transform
    ),
    batch_size=Nb,
    shuffle=True
)

test_loader = DataLoader(
    datasets.FashionMNIST(
        dir_data,
        train=False,
        transform=transform
    ),
    batch_size=Nb_test,
    shuffle=False
)


class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.net = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        return self.net(x)

model = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)


def train(model, loader):
    model.train()
    total_loss = 0

    for X, y in loader:
        optimizer.zero_grad()
        logits = model(X)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(loader)


def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for X, y in loader:
            logits = model(X)
            preds = torch.argmax(logits, dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)

    return correct / total

print("\n######## TREINAMENTO ########")
for epoch in range(epochs):
    loss = train(model, train_loader)
    acc = evaluate(model, test_loader)
    print(f"Época {epoch+1} - Loss: {loss:.4f} - Acurácia: {acc*100:.2f}%")


print("\n######## TESTE ########")

acc_final = evaluate(model, test_loader)
print(f"\nAcurácia no teste: {acc_final*100:.2f}%")



######## TREINAMENTO ########
Época 1 - Loss: 0.5076 - Acurácia: 84.91%
Época 2 - Loss: 0.3642 - Acurácia: 85.23%
Época 3 - Loss: 0.3231 - Acurácia: 87.47%
Época 4 - Loss: 0.3005 - Acurácia: 87.06%
Época 5 - Loss: 0.2806 - Acurácia: 87.74%
Época 6 - Loss: 0.2649 - Acurácia: 87.47%
Época 7 - Loss: 0.2531 - Acurácia: 88.60%
Época 8 - Loss: 0.2411 - Acurácia: 88.58%
Época 9 - Loss: 0.2297 - Acurácia: 88.77%
Época 10 - Loss: 0.2198 - Acurácia: 89.14%
Época 11 - Loss: 0.2111 - Acurácia: 89.61%
Época 12 - Loss: 0.2027 - Acurácia: 89.38%
Época 13 - Loss: 0.1922 - Acurácia: 89.22%
Época 14 - Loss: 0.1859 - Acurácia: 88.85%
Época 15 - Loss: 0.1772 - Acurácia: 89.15%
Época 16 - Loss: 0.1694 - Acurácia: 88.93%
Época 17 - Loss: 0.1618 - Acurácia: 89.01%
Época 18 - Loss: 0.1608 - Acurácia: 88.75%
Época 19 - Loss: 0.1516 - Acurácia: 89.27%
Época 20 - Loss: 0.1473 - Acurácia: 89.71%

######## TESTE ########

Acurácia no teste: 89.71%
